In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage.transform import resize
from scipy import ndimage
import json

import sys
sys.path.append('../aovek')

from aovek.network.network import YOLO

In [ ]:
def get_image_from_file(image_file):
    image_data = ndimage.imread(image_file).astype(float)

    if len(image_data.shape) != 3:
        image_data = np.repeat(image_data[:, :, np.newaxis],
                               3, axis=2)

    if image_data.shape != (288, 288, 3):
        image = resize(image_data,
                       output_shape=(288, 288),
                       mode='constant')
    else:
        image = image_data

    image = np.expand_dims(image, axis=0)

    image = normalize_image_from_0_to_1(image)

    return image
    

def normalize_image_from_0_to_1(image):
    normalized_image = image / 255

    return normalized_image

In [ ]:
def draw_rectangles(image, lables, prob):
    fig, ax = plt.subplots(1)
    ax.imshow(image)

    for label in lables:
        if prob:
            print(label[4])
        
        x = label[0] * 288
        y = label[1] * 288
        w = (label[2] - label[0]) * 288
        h = (label[3] - label[1]) * 288

        rect = Rectangle((x, y), w, h, linewidth=1,
                         edgecolor='r', facecolor='none')

        ax.add_patch(rect)

    plt.show()

In [ ]:
with open('./config.json') as config_file:
    config = json.load(config_file)

In [ ]:
def predict_image(image_file):
    image = get_image_from_file(image_file)
    all_labels = yolo.predict(image)
    all_labels = np.reshape(all_labels, (9 ** 2, 5))
    labels = yolo.predict_boxes(image)
    draw_rectangles(image[0], all_labels, False)
    draw_rectangles(image[0], labels, True)

In [ ]:
with tf.Session():
    yolo = YOLO(config)
    yolo.load_model()
    image_path = './dataset/cvpr10_multiview_pedestrians/train/03467.png'
    predict_image(image_path)